In [1]:
#NN Surrogate model class
import injector_surrogate_quads
from injector_surrogate_quads import *
import physics_gp

sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#Pytorch 
import numpy as np
import torch
import gpytorch
import botorch 

# BO for Minimizing Emittance*Bmag with 9 Variables (SQ, CQ, SOL, matching quads)

In [2]:
#load injector model
Model = Surrogate_NN()

Model.load_saved_model(model_path = '../models/', \
                       model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
Model.load_scaling()
Model.take_log_out = False

2022-06-27 17:22:50.344902: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Import design Twiss parameters (OTR2)

In [3]:
beamline_info = json.load(open('../configs/beamline_info.json'))
get_twiss0 = beamline_info['Twiss0']

# emit, beta, alpha
twiss0 = {'x': [get_twiss0[0], get_twiss0[2], get_twiss0[4]],
          'y': [get_twiss0[1], get_twiss0[3], get_twiss0[5]]}

beta0_x, alpha0_x = twiss0['x'][1], twiss0['x'][2]
beta0_y, alpha0_y = twiss0['y'][1], twiss0['y'][2]
print(twiss0['x'])
print(twiss0['y'])

[1e-06, 1.113081026, -0.0689403587]
[1e-06, 1.113021659, -0.07029489754]


## Objective Function

In [4]:
# convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

# input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale','CQ01:b1_gradient', 'SQ01:b1_gradient',
                 "QA01:b1_gradient", "QA02:b1_gradient", 
                 "QE01:b1_gradient", "QE02:b1_gradient", "QE03:b1_gradient", "QE04:b1_gradient"]
bounds = torch.tensor([[0.46, 0.485], [-0.02, 0.02], [-0.02, 0.02],
                       [-4, -1], [1, 4],
                       [-7,-1], [-1, 7],[-1, 7], [-7, 1]])

# output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

def evaluate(config): 
    """
    D is input space dimensionality
    :param config: input values of opt_var_names, torch.tensor, shape (1, D) 
    """
    #make input array of length model_in_list (inputs model takes)
    x_in = np.empty((1,len(Model.model_in_list)))

    #fill in reference point around which to optimize
    x_in[:,:] = np.asarray(ref_point[0])

    #set solenoid, CQ, SQ, matching quads to values from optimization step
    for i in range(config.size(dim=0)):
        x_in[:, Model.loc_in[opt_var_names[i]]] = config[i]

    #output predictions
    y_out = Model.pred_machine_units(x_in) 

    return -1*objective(y_out)[0]


def objective(y_out):
    # output is emittance * bmag 
    
    # geometric emittance in transverse plane
    out1 = y_out[:, Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    out2 = y_out[:, Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
    emit = np.sqrt(out1 * out2)
  
    sigma_x = y_out[:, Model.loc_out['sigma_x']] #grab sigma_x out of the model 
    sigma_y = y_out[:, Model.loc_out['sigma_y']] #grab sigma_y out of the model 
    
    # real beta and alpha 
    # NEEDS TO BE FIXED - currently assuming real alpha to be the same as design alpha 
    alpha_x, alpha_y = alpha0_x, alpha0_y
    beta_x, beta_y = (sigma_x**2) / out1, (sigma_y**2) / out2
    
    # bmag 
    bmag_x = 0.5 * ((beta0_x / beta_x) + (beta_x / beta0_x)) + 0.5 * ((alpha_x * np.sqrt(beta0_x / beta_x) - alpha0_x * np.sqrt(beta_x / beta0_x))**2)
    bmag_y = 0.5 * ((beta0_y / beta_y) + (beta_y / beta0_y)) + 0.5 * ((alpha_y * np.sqrt(beta0_y / beta_y) - alpha0_y * np.sqrt(beta_y / beta0_y))**2)
    bmag = np.sqrt(bmag_x * bmag_y)
    #print(f'bmag: {bmag} emit: {emit}') 
    
    return (emit * bmag)/1e-6 # in um units 
    #return np.sqrt(out1*out2)/1e-6 # in um units

## Gaussian Regression & Acquisition Function

In [5]:
def get_BO_point(x, f, bounds, beta=2.5, mean_module=None, input_transform=None, outcome_transform=None, precision = None, phys=False):
    """
    function that trains a GP model of data and returns the next observation point using UCB
    D is input space dimensionality
    N is number of samples

    :param x: input points data, torch.tensor, shape (N,D)
    :param f: output point data, torch.tensor, shape (N,1)
    :param bounds: input space bounds, torch.tensor, shape (2,D)
    :param precision: precision matrix used for RBF kernel (must be PSD), torch.tensor, (D,D)
    :param beta: UCB optimization parameter, float
    :return x_candidate, model: next observation point and gp model w/observations
    """
    
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    
    if phys == True:
        gp = physics_gp.PhysicsExactGPModel(x, f.flatten(), likelihood, precision)
    else:
        gp = botorch.models.SingleTaskGP(x, f, likelihood=likelihood, 
                                         mean_module=mean_module, 
                                         outcome_transform=outcome_transform, 
                                         input_transform=input_transform)
        
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(gp.likelihood, gp)
    
    # fit GP hyperparameters
    botorch.fit.fit_gpytorch_model(mll)

    # do UCB acquisition
    UCB = botorch.acquisition.UpperConfidenceBound(gp, beta=beta)
    candidate, acq_value = botorch.optim.optimize_acqf(UCB,
                                                       bounds=bounds,
                                                       q=1,
                                                       num_restarts=20,
                                                       raw_samples=20)
    return candidate, gp

## Set up initial training samples

In [6]:
#create initial samples within specified bounds
n_samples = 5
n_var = 9

train_x = torch.zeros((n_samples, n_var)) 
for i in range(n_var):
    train_x[:,i] = torch.as_tensor(np.random.uniform(bounds[i,0],bounds[i,1],(n_samples,)))
print(train_x)

train_y = torch.tensor([evaluate(vars) for vars in train_x]).reshape(-1,1)
print(train_y)

#transformer 
transformer_x = botorch.models.transforms.input.Normalize(n_var, bounds = bounds.transpose(0,1))
transformer_y = botorch.models.transforms.outcome.Standardize(1)

normed_bounds = torch.cat((torch.zeros(1,n_var), torch.ones(1,n_var)), 0)

tensor([[ 4.7461e-01,  1.5283e-02, -1.6077e-02, -1.5335e+00,  1.9540e+00,
         -5.0741e+00,  3.9297e+00,  4.6886e+00, -1.9923e+00],
        [ 4.6882e-01,  9.6068e-03,  1.8685e-03, -3.0921e+00,  2.5592e+00,
         -4.0247e+00,  4.8338e+00,  2.4705e+00,  7.8012e-01],
        [ 4.6593e-01,  1.5669e-02,  1.0070e-02, -1.6309e+00,  2.2629e+00,
         -2.6205e+00,  6.0606e+00,  2.5648e+00,  8.5917e-01],
        [ 4.7787e-01,  1.3221e-02,  8.9130e-03, -3.6445e+00,  2.1859e+00,
         -6.8279e+00,  2.6395e+00, -4.4129e-01,  3.9444e-01],
        [ 4.7177e-01, -8.3972e-03, -8.9095e-03, -1.2805e+00,  3.4294e+00,
         -1.8193e+00,  3.4236e+00,  1.2174e+00, -2.3277e+00]])


2022-06-27 17:22:50.776301: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-27 17:22:50.782908: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1996350000 Hz


tensor([[ -3.0934],
        [ -1.9115],
        [-15.7657],
        [ -1.7430],
        [ -3.4508]], dtype=torch.float64)


## Bayesian Optimization

In [7]:
def BayesianOptimization(train_x, train_y, n_steps, prior = None, phys = False):
    best_y = torch.max(train_y)
    for i in range(n_steps):
        if (phys == True):
            normed_train_x = transformer_x.forward(train_x) 
            normed_train_y = transformer_y.forward(train_y)[0]

            x_new, model = get_BO_point(normed_train_x, normed_train_y, 
                                        bounds=normed_bounds, 
                                        phys=phys)

            train_x = torch.cat((train_x, transformer_x.untransform(x_new)))
            new_y = torch.tensor(evaluate(train_x[-1])).reshape(1,1)
            train_y = torch.cat((train_y, new_y))
        else:
            x_new, model = get_BO_point(train_x, train_y, 
                                        bounds=bounds.transpose(0,1), 
                                        mean_module=prior, 
                                        input_transform=transformer_x, 
                                        outcome_transform=transformer_y, 
                                        phys=phys)

            train_x = torch.cat((train_x, x_new))
            new_y = torch.tensor(evaluate(train_x[-1])).reshape(1,1)
            train_y = torch.cat((train_y, new_y))

        if (new_y > best_y):
            best_y = new_y
            color = '\033[95m', '\033[0m'
        else: 
            color = '\u001b[30m', '\033[0m'

        print("iter     target       SOL        CQ        SQ        QA1        QA2        Q1        Q2        Q3        Q4")
        print(f'{color[0]}{i+1}      {train_y[-1,0]:.5f}   {train_x[-1,0]:.5f}   {train_x[-1,1]:.5f}   {train_x[-1,2]:.5f}   {train_x[-1,3]:.5f}   {train_x[-1,4]:.5f}   {train_x[-1,5]:.5f}   {train_x[-1,6]:.5f}   {train_x[-1,7]:.5f}   {train_x[-1,8]:.5f}{color[1]}')

In [8]:
# constant prior mean
BayesianOptimization(train_x, train_y, 20, prior = None, phys = False) 

iter     target       SOL        CQ        SQ        QA1        QA2        Q1        Q2        Q3        Q4
1      -6.89511   0.47371   0.00818   -0.00208   -3.69794   2.52588   -5.59964   3.44617   1.74433   0.28771
iter     target       SOL        CQ        SQ        QA1        QA2        Q1        Q2        Q3        Q4
2      -3.68145   0.48113   0.02000   0.02000   -3.40272   1.77249   -7.00000   2.21540   -1.00000   0.82283
iter     target       SOL        CQ        SQ        QA1        QA2        Q1        Q2        Q3        Q4
3      -3.31950   0.47306   0.00980   0.01940   -3.42855   2.49038   -7.00000   2.62067   -1.00000   -1.03785
iter     target       SOL        CQ        SQ        QA1        QA2        Q1        Q2        Q3        Q4
4      -4.28714   0.48351   0.00792   0.01347   -4.00000   2.36343   -7.00000   1.30827   -1.00000   -0.62954
iter     target       SOL        CQ        SQ        QA1        QA2        Q1        Q2        Q3        Q4
5      -1.29313   0.47

### use NN as prior mean

In [9]:
# model 
import torch.nn as nn
class NN_prior(nn.Module):
    def __init__(self):
        super(NN_prior, self).__init__()
        
        hidden_size = 20
        self.network = nn.Sequential(nn.Linear(n_var, hidden_size), 
                                     nn.Tanh(), 
                                     nn.Linear(hidden_size, hidden_size), 
                                     nn.Tanh(),
                                     nn.Linear(hidden_size, 1))
        
    def forward(self, x):
        x = self.network(x)
        return x 

# create prior mean
from gpytorch.means.mean import Mean
def CustomMean(Mean):
    def __init__(self, name):
        self.NN_model = NN_prior()
        self.NN_model.load_state_dict(torch.load('./results/' + name + '.pth'))
        self.NN_model.eval()
    
    def forward(self, x):
        return self.NN_model(x)

In [10]:
NN_prior_mean = CustomMean('model1')
BayesianOptimization(train_x, train_y, 20, prior = NN_prior_mean, phys = False)

iter     target       SOL        CQ        SQ        QA1        QA2        Q1        Q2        Q3        Q4
1      -6.53102   0.47370   0.00820   -0.00194   -3.70371   2.55934   -5.57111   3.42076   1.71289   0.22847
iter     target       SOL        CQ        SQ        QA1        QA2        Q1        Q2        Q3        Q4
2      -3.51279   0.48054   0.02000   0.02000   -3.42665   1.66653   -7.00000   2.58948   -1.00000   1.00000
iter     target       SOL        CQ        SQ        QA1        QA2        Q1        Q2        Q3        Q4
3      -1.78472   0.47222   0.01046   0.01860   -3.24641   2.61971   -7.00000   3.22016   -1.00000   0.58399
iter     target       SOL        CQ        SQ        QA1        QA2        Q1        Q2        Q3        Q4
4      -1.60678   0.47900   0.00855   0.01897   -3.50058   2.73528   -7.00000   1.28351   -1.00000   -0.97003
iter     target       SOL        CQ        SQ        QA1        QA2        Q1        Q2        Q3        Q4
5      -4.52559   0.471

In [11]:
"""
#define acquisition function
from botorch.acquisition.analytic import UpperConfidenceBound, ExpectedImprovement
from botorch.optim import optimize_acqf

#optimize
n_steps = 45
for i in range(n_steps):
    #best_normed_y = torch.max(normed_train_y)
    UCB = UpperConfidenceBound(gp, beta=2.5)
    #EI = ExpectedImprovement(gp, best_normed_y)

    bounds = torch.cat((torch.zeros(1,3), torch.ones(1,3)), 0)
    candidate, acq_value = optimize_acqf(UCB, bounds = bounds, num_restarts = 20, q = 1, raw_samples = 20)

    train_x = torch.cat((train_x, transformer_x.backward(candidate)))
    normed_train_x = transformer_x.forward(train_x)

    new_y = torch.tensor(evaluate(train_x[-1][0], train_x[-1][1], train_x[-1][2])).reshape(1,1)
    train_y = torch.cat((train_y, new_y))
    
    print("iteration        target         varx         vary         varz")
    print(f'{i+1}              {train_y[-1][0]:.5f}      {train_x[-1][0]:.5f}      {train_x[-1][1]:.5f}      {train_x[-1][2]:.5f}')
    print(torch.max(train_y))
    
    transformer_y = transformer.Transformer(train_y, 'standardize')
    normed_train_y = transformer_y.forward(train_y)

    gp = SingleTaskGP(normed_train_x, normed_train_y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_model(mll);
"""

'\n#define acquisition function\nfrom botorch.acquisition.analytic import UpperConfidenceBound, ExpectedImprovement\nfrom botorch.optim import optimize_acqf\n\n#optimize\nn_steps = 45\nfor i in range(n_steps):\n    #best_normed_y = torch.max(normed_train_y)\n    UCB = UpperConfidenceBound(gp, beta=2.5)\n    #EI = ExpectedImprovement(gp, best_normed_y)\n\n    bounds = torch.cat((torch.zeros(1,3), torch.ones(1,3)), 0)\n    candidate, acq_value = optimize_acqf(UCB, bounds = bounds, num_restarts = 20, q = 1, raw_samples = 20)\n\n    train_x = torch.cat((train_x, transformer_x.backward(candidate)))\n    normed_train_x = transformer_x.forward(train_x)\n\n    new_y = torch.tensor(evaluate(train_x[-1][0], train_x[-1][1], train_x[-1][2])).reshape(1,1)\n    train_y = torch.cat((train_y, new_y))\n    \n    print("iteration        target         varx         vary         varz")\n    print(f\'{i+1}              {train_y[-1][0]:.5f}      {train_x[-1][0]:.5f}      {train_x[-1][1]:.5f}      {train_x[-